In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dateutil.parser
import seaborn as sns
from IPython.display import Image
import os

fil_path = "data/PLUTO17v1.1/BK2017V11.csv"

''' Settings '''
sns.set_style("white")
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.precision', 3)

''' Load and concatinate CSV files into single dataframe'''

# Brooklyn PLUTO data
csv_names = ['../data/pluto-v17.1.1/BK2017V11.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path,index_col=0)
    )
df_pluto = pd.concat(files) 

# 2013 data
csv_names = ['../data/pluto-v13v2/BK.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path,index_col=0)
    )
df_pluto_2016 = pd.concat(files) 

# Affordible housing data
csv_names = ['../data/data.nyc.gov/Housing_New_York_Units_by_Building.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path,index_col=0)
    )
df_housing = pd.concat(files) 

# Income census data
csv_names = ['../data/census/ACS_16_5YR_B25064_with_ann.csv']
files = []
for path in csv_names:
    files.append(
        pd.read_csv(path,index_col=0)
    )
df_census = pd.concat(files) 

df_pluto = df_pluto.reset_index()
df_pluto_2016 = df_pluto_2016.reset_index()
df_pluto_2016 = df_pluto_2016[['LandUse','BBL']]
df_pluto_2016['LandUse2016'] = df_pluto_2016['LandUse']
df_pluto_2016 = df_pluto_2016[df_pluto_2016.columns.difference(['LandUse'])]

df_todos = pd.merge(df_pluto, df_pluto_2016, on='BBL')
df_todos[df_todos['Address'] == '781 UNION STREET']


/Users/justinblinder/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,20,22,23,24,25,26,64,65,80) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/justinblinder/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,15,17,18,20,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCenterDistrict,HealthArea,SanitBoro,SanitDistrict,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptLand,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version,LandUse2016
20444,BK,953,55,306,131.0,3001.0,13.0,39.0,11215.0,L105,78.0,38.0,4200.0,3.0,6.0,4D,781 UNION STREET,R6A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,C6,2.0,0,NaN,781 UNION ST OWNERS C,2375,6500,0,6500,0,0,0,0,0,0,2,1,4.0,8,8,25.0,95.0,25.0,65.0,NaN,0.0,N,5.0,2.0,1975,121598,1975,7080,1912,0,0,NaN,NaN,2.74,3.0,0.0,3.0,3,3009530055,0,131,990233.0,185521.0,16c,NaN,306 038,30401.0,NaN,0.0,NaN,1,NaN,NaN,17v1.1,2.0


## Did zone code change between n years?

In [11]:
def check_zone_change(row):
    if row['LandUse'] == row['LandUse2016']:
        return 0
    else:
        return 1

# Create feature based on whether block contains a new building
df_todos['ZoneCodeChanged'] = df_todos.apply(check_zone_change,axis=1)
df_todos['ZoneCodeChanged'].value_counts()
df_todos[df_todos['ZoneCodeChanged']==1]['LandUse'].value_counts()

4.0     1994
2.0     1177
10.0     914
1.0      726
11.0     613
5.0      554
3.0      381
6.0      200
8.0      169
7.0       33
9.0       31
Name: LandUse, dtype: int64

In [12]:

df_todos[df_todos['ZoneCodeChanged']==1]

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCenterDistrict,HealthArea,SanitBoro,SanitDistrict,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptLand,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version,LandUse2016,ZoneCodeChanged
0,BK,1,1,302,21.00,NaN,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,JOHN STREET,M3-1,M1-4/R8A,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,Y,V1,11.0,1,P,BROOKLYN BRIDGE PARK,151930,0,0,0,0,0,0,0,0,0,4,0,0.0,0,0,206.25,337.42,0.00,0.00,NaN,0.0,Y,5.0,5.0,1371105,1371105,0,0,0,0,0,NaN,NaN,0.00,0.00,2.0,0.0,3,3000010001,0,21,NaN,NaN,12d,NaN,302 007,30101.0,NaN,3.000e+09,11/26/2013,1,1.0,1.0,17v1.1,10.0,1
1,BK,1,50,302,21.00,2000.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,10 JAY STREET,M1-4/R8A,M3-1,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,Y,O6,5.0,0,NaN,SAFDI PLAZA REALTY IN,19682,163894,163894,0,147505,16389,0,0,0,0,2,1,10.0,0,2,109.08,207.25,88.00,195.00,NaN,1.0,Y,3.0,0.0,834300,10156950,0,0,1920,1994,2015,DUMBO Historic District,NaN,8.33,5.40,2.0,6.5,3,3000010050,0,21,9.878e+05,195989.0,12d,NaN,302 007,30101.0,E-231,0.000e+00,NaN,1,1.0,1.0,17v1.1,6.0,1
9,BK,6,35,302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,NaN,NaN,0.00,0.00,0.0,0.0,3,3000060035,0,0,NaN,NaN,12d,NaN,NaN,NaN,NaN,NaN,NaN,5,1.0,1.0,17v1.1,NaN,1
13,BK,7,1,302,21.00,2008.0,13.0,33.0,11201.0,E205,84.0,36.0,1000.0,3.0,2.0,1B,111 PLYMOUTH STREET,M3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,Q1,9.0,0,C,NYC PARKS,80873,0,0,0,0,0,0,0,0,0,7,1,0.0,0,0,128.75,363.00,0.00,0.00,NaN,0.0,Y,5.0,5.0,679140,679140,679140,679140,0,0,0,NaN,NaN,0.00,0.00,2.0,0.0,3,3000070001,0,21,9.873e+05,195983.0,12d,NaN,302 006,30101.0,NaN,0.000e+00,NaN,1,1.0,1.0,17v1.1,7.0,1
21,BK,18,1,302,21.00,2002.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,135 PLYMOUTH STREET,M1-4/R8A,NaN,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,N,D5,3.0,0,NaN,DON BEN R CO,41250,211386,37360,174026,0,37360,0,0,0,0,2,1,7.0,100,102,206.25,200.00,156.00,48.00,NaN,0.0,N,3.0,5.0,514350,7785450,0,0,1900,2014,0,DUMBO Historic District,NaN,5.12,5.40,2.0,6.5,3,3000180001,0,21,9.876e+05,195811.0,12d,NaN,302 006,30101.0,E-231,0.000e+00,NaN,1,1.0,1.0,17v1.1,6.0,1
22,BK,19,1,302,21.00,2003.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,20 JAY STREET,M1-4/R8A,NaN,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,N,O6,5.0,0,P,20 JAY LLC,41400,500000,500000,0,418182,22576,40000,19242,0,0,2,1,11.0,0,147,200.00,207.00,200.00,207.00,NaN,0.0,Y,3.0,5.0,972000,40980600,0,6820065,1911,1999,0,DUMBO Historic District,NaN,12.08,5.40,2.0,6.5,3,3000190001,0,21,9.878e+05,195800.0,12d,NaN,302 007,30101.0,E-231,0.000e+00,NaN,1,1.0,1.0,17v1.1,6.0,1
27,BK,20,29,302,21.00,3008.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,195 PLYMOUTH STREET,M1-4/R7A,NaN,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,N,O5,5.0,0,NaN,SHEVA REALTY CORP,22300,67310,67310,0,28940,21970,0,0,16400,0,2,5,6.0,0,3,100.00,223.00,50.00,170.00,NaN,0.0,Y,5.0,5.0,301050,3915900,0,0,1910,0,0,DUMBO Historic District,NaN,3.02,4.00,2.0,4.0,3,3000200029,0,21,9.884e+05,195726.0,12d,NaN,302 007,30101.0,E-231,0.000e+00,NaN,1,NaN,NaN,17v1.1,6.0,1
41,BK,27,40,302,21.00,1010.0,13.0,33.0,11201.0,E205,84.0,36.0,1000.0,3.0,2.0,1B,26 WASHINGTON STREET,C6-2A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,D7,4.0,0,NaN,30 WASHINGTON RENTAL,11882,87129,2201,84928,0,2201,0,0

## Did zone code change from vacant lot?

In [13]:
def check_vacancy(row):
    if row['ZoneCodeChanged']==1:
        if row['LandUse2016'] == 11.0:
            return 1
        else:
            return 0
    else:
        return 0

# Create feature based on whether block contains a new building
df_todos['ZoneCodeWasVacant'] = df_todos.apply(check_vacancy,axis=1)
df_todos['ZoneCodeWasVacant'].value_counts()

0    275114
1      1022
Name: ZoneCodeWasVacant, dtype: int64

## Does the current land use match mixed use or condo? And was it previously another zoning code? 

```
CODES
DECODES
01 One &Two Family Buildings
02 Multi-Family Walk-Up Buildings
03 Multi-Family Elevator Buildings
04 Mixed Residential & Commercial Buildings
05 Commercial & Office Buildings
06 Industrial & Manufacturing
07 Transportation & Utility
08 Public Facilities & Institutions
09 Open Space & Outdoor Recreation
10 Parking Facilities
11 Vacant Land
```

In [14]:
def check_vacancy(row):
    if row['ZoneCodeChanged'] == 1:
        if (row['LandUse'] == 2.0 or row['LandUse'] == 3.0 or row['LandUse'] == 4.0) and (row['LandUse2016'] is not  2.0 or row['LandUse2016'] is not 3.0 or row['LandUse'] is not 4.0):
            return 1
        else:
            return 0
    else:
        return 0

# Create feature based on whether block contains a new building
df_todos['ZoneCodeBecameCondo'] = df_todos.apply(check_vacancy,axis=1)

#df_todos['ZoneCodeChanged'].value_counts()
#df_todos[df_todos['ZoneCodeChanged']==1]['LandUse'].value_counts()
df_todos[df_todos['ZoneCodeBecameCondo']==1]

,Borough,Block,Lot,CD,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCenterDistrict,HealthArea,SanitBoro,SanitDistrict,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptLand,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLAG,PFIRM15_FLAG,Version,LandUse2016,ZoneCodeChanged,ZoneCodeWasVacant,ZoneCodeBecameCondo
21,BK,18,1,302,21.00,2002.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,135 PLYMOUTH STREET,M1-4/R8A,NaN,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,N,D5,3.0,0,NaN,DON BEN R CO,41250,211386,37360,174026,0,37360,0,0,0,0,2,1,7.0,100,102,206.25,200.00,156.00,48.00,NaN,0.0,N,3.0,5.0,514350,7785450,0,0,1900,2014,0,DUMBO Historic District,NaN,5.12,5.40,2.0,6.5,3,3000180001,0,21,9.876e+05,195811.0,12d,NaN,302 006,30101.0,E-231,0.000e+00,NaN,1,1.0,1.0,17v1.1,6.0,1,0,1
41,BK,27,40,302,21.00,1010.0,13.0,33.0,11201.0,E205,84.0,36.0,1000.0,3.0,2.0,1B,26 WASHINGTON STREET,C6-2A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,D7,4.0,0,NaN,30 WASHINGTON RENTAL,11882,87129,2201,84928,0,2201,0,0,0,0,2,1,7.0,94,95,154.63,83.58,154.63,83.58,NaN,0.0,Y,5.0,5.0,486000,10550250,0,0,1930,2012,0,DUMBO Historic District,NaN,7.33,6.02,6.0,6.5,3,3000270040,0,21,9.871e+05,195567.0,12d,NaN,302 006,30101.0,E-84,3.000e+09,1/16/1998,1,1.0,1.0,17v1.1,8.0,1,0,1
49,BK,30,1,302,21.00,2004.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,55 PEARL STREET,M1-4/R8A,NaN,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,N,S1,4.0,0,P,KMM LLC,965,3676,270,3406,0,270,0,0,0,0,2,1,4.0,1,2,19.17,50.42,18.00,50.42,NaN,2.0,Y,3.0,5.0,50405,71586,0,0,2014,0,0,DUMBO Historic District,NaN,3.81,5.40,2.0,6.5,3,3000300001,0,21,9.877e+05,195490.0,12d,NaN,302 007,30101.0,E-231,3.000e+09,12/2/2013,1,NaN,NaN,17v1.1,7.0,1,0,1
60,BK,31,1,302,21.00,3009.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,39 JAY STREET,M1-4/R8A,M1-4/R7A,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,Y,D7,4.0,0,NaN,201 WATER STREET LLC,27250,0,0,0,0,0,0,0,0,0,0,0,0.0,74,75,200.00,155.00,0.00,0.00,NaN,0.0,Y,3.0,5.0,613350,4077450,0,2355478,0,0,0,DUMBO Historic District,NaN,0.00,5.40,2.0,6.5,3,3000310001,0,21,NaN,NaN,12d,NaN,NaN,NaN,E-231,3.000e+09,1/20/2017,2,NaN,NaN,17v1.1,6.0,1,0,1
65,BK,31,7503,302,21.00,3009.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,1.0,2.0,NaN,205 WATER STREET,M1-4/R7A,M1-4/R8A,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,Y,R4,3.0,1,NaN,NaN,21740,82761,11977,70784,0,0,11020,957,0,0,2,1,9.0,65,65,89.58,200.00,115.00,200.00,NaN,1.0,Y,0.0,5.0,1221761,10042601,135085,6107650,2010,0,0,NaN,NaN,3.81,4.00,2.0,4.0,3,3000317503,3187,21,9.882e+05,195549.0,12d,NaN,320 007,30101.0,NaN,3.000e+09,5/8/2012,1,NaN,NaN,17v1.1,4.0,1,0,1
108,BK,35,10,302,21.00,1005.0,13.0,33.0,11201.0,E205,84.0,36.0,1000.0,3.0,2.0,1B,9 OLD FULTON STREET,M2-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,S4,4.0,0,P,"9 OLD FULTON, LLC",1387,4926,1836,3090,0,1836,0,0,0,0,2,1,4.0,4,5,22.67,61.58,22.00,56.00,NaN,0.0,Y,5.0,5.0,33750,310590,0,0,2015,2010,0,Fulton Ferry Historic District,NaN,3.55,0.00,2.0,0.0,3,3000350010,0,21,9.860e+05,195379.0,12d,NaN,302 005,30101.0,NaN,0.000e+00,NaN,1,NaN,NaN,17v1.1,NaN,1,0,1
126,BK,41,2,302,21.00,3016.0,13.0,33.0,11201.0,L118,84.0,36.0,1000.0,3.0,2.0,1B,67 JAY STREET,M1-4/R8A,NaN,NaN,NaN,NaN,NaN,MX-2,NaN,NaN,NaN,N,K4,4.0,0,NaN,JAYAT REALTY CORP.,6028,8775,8775,0,0,6028,0,2747,0,0,2,4,3.0,2,3,74.58,87.83,74.58,87.83,E,0.0,Y,5.0,2.0,54000,514800,0,0,1910,1986,0,NaN,NaN,1.46,5.40,2.0,6.5,3,3000410002,0,21,9.880e+05,195275.0,12d,NaN,302 007,30101.0,E-231,0.000e+00,NaN,1,NaN,NaN,17v1.1,5.0,1,0,1
128,BK,41,7

In [15]:
df_pluto = df_todos

> Drop columns with lots of nulls or no unique values

In [16]:
drop_cols = ['SanitSub',
            'Ext',
            'OwnerType',
            'Overlay1',
            'SPDist1',
            'PFIRM15_FLAG',
            'APPDate',
            'HistDist',
            'FIRM07_FLAG',
            'ZoneDist2',
            'ZMCode',
            'EDesigNum',
            'LtdHeight',
            'Landmark',
            'ZoneDist3',
            'Overlay2',
            'SPDist2',
            'ZoneDist4',
            'SPDist3',
            'Address',
            'OwnerName',
            'APPDate',
            'PLUTOMapID',
            'FIRM07_FLAG',
            'PFIRM15_FLAG',
            'Version',
            'ZoneMap',
            'Borough',
            'HealthCenterDistrict',
            'HealthArea']
df_pluto = df_pluto[df_pluto.columns.difference(drop_cols)]

In [17]:
df_pluto.to_pickle('../data/pkl/df_pluto-2013-2017.pkl')